# Analyzing Code Performance

### Timing Your Code

In [1]:
def slow_way_to_calculate_mode(list_of_numbers):
    result_dict = {}
    for i in list_of_numbers:
        if i not in result_dict:
            result_dict[i] = 1
        else:
            result_dict[i] += 1

    mode_vals = []
    max_frequency = max(result_dict.values())
    for key, value in result_dict.items():
        if value == max_frequency:
            mode_vals.append(key)

    return mode_vals

In [3]:
slow_way_to_calculate_mode([4, 5, 5, 6])

[5]

In [4]:
import numpy as np

random_integers = np.random.randint(1, 1_000_000, 1_000_000)

In [5]:
import time

start = time.time()
slow_way_to_calculate_mode(random_integers)
end = time.time()

print(end - start)

0.4956214427947998


In [6]:
%%timeit
slow_way_to_calculate_mode(random_integers)

457 ms ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In this case, timeit looped through the code 10 times, and it repeated the set of 10 loops 7 times, then returned the summary statistics of those runs. timeit defaults to the number of runs and loops that will fit into a 2-second window, but you can specify the number of runs using the -r flag and the number of loops using the -n flag.
If you’re using a standalone script, you can use timeit as shown in this example:

In [13]:
import numpy as np
import timeit

random_integers = np.random.randint(1, 100_000, 100_000)

def slow_way_to_calculate_mode(list_of_numbers):
    result_dict = {}
    for i in list_of_numbers:
        if i not in result_dict:
            result_dict[i] = 1
        else:
            result_dict[i] += 1

    mode_vals = []
    max_frequency = max(result_dict.values())
    for key, value in result_dict.items():
        if value == max_frequency:
            mode_vals.append(key)

    return mode_vals

mode_timer = timeit.Timer(stmt="slow_way_to_calculate_mode(random_integers)",
                          setup="from __main__ import slow_way_to_calculate_mode, random_integers")

time_taken = mode_timer.timeit(number=10)

print(f"Execution time: {time_taken} seconds")


Execution time: 0.363119000000097 seconds


But is this code any good? Is this fast or slow? Because this is just an example without real-world requirements for speed, I’ll compare it with another way of calculating the mode and see if that improves the performance. Here’s another way to carry out the same calculation:

In [15]:
from collections import Counter

def mode_using_counter(list_of_numbers):
    c = Counter(list_of_numbers)
    return c.most_common(1)[0][0]

And again using timeit to measure how long it takes to run:

In [16]:
%%timeit
mode_using_counter(random_integers)

20.5 ms ± 792 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As you can see, this version of the code takes much less time to run. Instead of an average of approximately 267 ms, it takes only approximately 22 ms. The standard deviation is also much lower, which helps guarantee an upper bound to the runtime.

## Profiling Your Code

### `cProfile`

In [17]:
from collections import Counter
import numpy as np

In [18]:
def mode_using_counter(n_integers):
    random_integers = np.random.randint(1, 100000, n_integers)
    c = Counter(random_integers)
    return c.most_common(1)[0][0]

In [19]:
mode_using_counter(10000000)

57076

In [20]:
%%timeit
mode_using_counter(10000000)

3.16 s ± 327 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%prun
mode_using_counter(10000000)

         23 function calls in 3.569 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.373    3.373    3.373    3.373 {built-in method _collections._count_elements}
        1    0.181    0.181    0.181    0.181 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
        1    0.008    0.008    0.008    0.008 {built-in method builtins.max}
        1    0.007    0.007    3.569    3.569 <string>:1(<module>)
        1    0.001    0.001    0.001    0.001 fromnumeric.py:2974(_prod_dispatcher)
        1    0.000    0.000    3.563    3.563 3568474488.py:1(mode_using_counter)
        1    0.000    0.000    3.373    3.373 __init__.py:640(update)
        1    0.000    0.000    3.569    3.569 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 fromnumeric.py:71(_wrapreduction)
        1    0.000    0

The tottime column in this output shows where the computer spent most of the time when running this code: in the built-in method _collections._count_elements function, which is the Counter function. The next most time-consuming part was method 'randint' of 'numpy.random.mtrand.RandomState' objects, which is the step that created the list of random numbers. All the other steps took very little time. The disadvantage of using cProfile is that you need to map each of these function calls back to lines within your code.

In [22]:
%load_ext snakeviz

In [27]:
%%snakeviz
mode_using_counter(1000000)

 
*** Profile stats marshalled to file 'C:\\Users\\anpande\\AppData\\Local\\Temp\\tmp2bvs7b3t'.
Embedding SnakeViz in this document...
<function display at 0x00000225F7206DD0>


`line_profiler`

In [28]:
%load_ext line_profiler

In [29]:
%lprun -f mode_using_counter mode_using_counter(10000000)

Timer unit: 1e-07 s

Total time: 2.80263 s
File: C:\Users\anpande\AppData\Local\Temp\ipykernel_22140\3568474488.py
Function: mode_using_counter at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mode_using_counter(n_integers):
     2         1    1108546.0    1e+06      4.0      random_integers = np.random.randint(1, 100000, n_integers)
     3         1   26855111.0    3e+07     95.8      c = Counter(random_integers)
     4         1      62691.0  62691.0      0.2      return c.most_common(1)[0][0]

You can profile your code’s memory usage as well as the time it takes to run. Memory usage is something you also may need to optimize, depending on your code’s requirements. It’s important to consider this as the size of your data increases, because you may hit the upper limits of the hardware you are using. Additionally, the CPU has to work harder to manage memory. This could increase the runtime of your code if the CPU is spending too much time managing the memory instead of executing code.


## Time Complexity

In [8]:
def weighted_mean(list_of_numbers, weights):
    running_total = 0
    for i in range(len(list_of_numbers)):
        running_total += (list_of_numbers[i] * weights[i])
    return (running_total/sum(weights))

In [9]:
def covariance_fast(X, Y):
    avg_X = sum(X) / len(X)
    avg_Y = sum(Y) / len(Y)

    result = 0
    for i in range(len(X)):
        result += (X[i] - avg_X) * (Y[i] - avg_Y)

    return result / len(X)

In [4]:
import numpy as np
X = np.random.randint(1, 1000, 1000)
Y = np.random.randint(1, 1000, 1000)

In [5]:
%%timeit
covariance_fast(X, Y)

723 µs ± 52.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
def covariance(X, Y):
    cov_sum = 0
    for i in range(len(X)):
        for j in range(len(Y)):
            cov_sum += 0.5 * (X[i] - X[j]) * (Y[i] - Y[j])
    return cov_sum / (len(X) ** 2)

In [7]:
%%timeit
covariance(X, Y)

3.17 s ± 563 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Code to generate Figure 2.3

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
n = np.linspace(1, 10, 1000)
line_names = [
    "Constant",
    "Linear",
    "Quadratic",
    "Exponential",
    "Logarithmic",
    "N log N",
]
big_o = [np.ones(n.shape), n, n**2, 2**n, np.log(n), n * (np.log(n))]

fig, ax = plt.subplots()
fig.set_facecolor("white")

ax.set_ylim(0, 50)
for i in range(len(big_o)):
    ax.plot(n, big_o[i], label=line_names[i])
ax.set_ylabel("Relative Runtime")
ax.set_xlabel("Input Size")
ax.legend()
fig.savefig("ch02_02.png", bbox_inches="tight")